In [1]:
import geopandas as gpd
from IPython.display import IFrame
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import scipy.stats as ss
import seaborn as sn

<a id="data" />

## Data Cleaning Process

### Data Sources:
* [Map - AIDSVu](https://map.aidsvu.org/map)
* [American FactFinder](https://factfinder.census.gov) we used the DP03 "SELECTED ECONOMIC CHARACTERISTICS" table from the 2013-2017 American Community Survey 5-Year Estimates


### Labeling Columns
```
VC03 -> LaborForceParticipationRate
VC06 -> EmploymentRate
VC07 -> UnemploymentRate
VC16 -> FemaleLaborForceParticipation
VC28 -> CommutingToWork-DrivingAlone
VC29 -> CommutingToWork-Carpool
VC30 -> CommutingToWork-PublicTransit
VC31 -> CommutingToWork-Walked
VC33 -> CommutingToWork-Home
VC67 -> PrivateWageSalary
VC75 -> Income<10,000
VC76 -> Income10k-15k
VC77 -> Income15k-25k
VC78 -> Income25k-35k
VC79 -> Income35k-50k
VC80 -> Income50k-75k
VC81 -> Income75k-100k
VC82 -> Income100k-150k
VC83 -> Income150k-200k
VC84 -> Income200K+
VC85 -> MedianHouseholdIncome
VC86 -> MeanHouseholdIncome
VC89 -> HouseholdsWithEarnings
VC90 -> HouseholdsMeanEarnings
VC91 -> HouseholdsWSS
VC92 -> HouseholdsMeanSSIncome
VC99 -> HouseholdsWPublicAssistanceIncome
VC100 -> HouseholdsMeanPublicAssistanceIncome
VC101 -> HouseholdsWFoodStampSNAPBenefits
VC104 -> FamiliesIncome<10k
VC105 -> FamiliesIncome10k-15k
VC106 -> FamiliesIncome15k-25k
VC107 -> FamiliesIncome25k-35k
VC108 -> FamiliesIncome35k-50k
VC109 -> FamiliesIncome50k-75k
VC110 -> FamiliesIncome75k-100k
VC111 -> FamiliesIncome100k-150k
VC112 -> FamiliesIncome150k-200k
VC113 -> FamiliesIncome200k+
VC114 -> FamiliesMedianIncome
VC115 -> FamiliesMeanIncome
VC118 -> FamiliesPerCaptiaIncome
VC121 -> NonFamilyHouseholdsMedianIncome
VC122 -> NonFamilyHouseholdsMeanIncome
VC131 -> HealthInsuranceCoverage
VC132 -> HealthInsuranceCoverage-Private
VC133 -> HealthInsuranceCoverage-Public
VC161 -> FamiliesBelowPovertyLevel
VC164 -> MarriedCouplesBelowPovertyLevel
VC171 -> PeopleIncomeBelowPovertyLevel
VC172 -> ChildrenUnderPovertyLevel
```

<a id="census" />

## Chicago at the Census Tract Level

In [2]:
# Census Tract Data for Chicago
Chicago_data = pd.read_csv("./data/census/DP03IL-CensusTract-Cleaned.csv")
Chicago_data.head()

,GEO.id,GEOID,GEO.display-label,LaborForceParticipationRate,LaborForceParticipationRate-MOE,EmploymentRate,EmploymentRate-MOE,UnemploymentRate,UnemploymentRate-MOE,FemaleLaborForceParticipation,...,HealthInsuranceCoverage-Public,HealthInsuranceCoverage-Public-MOE,FamiliesBelowPovertyLevel,FamiliesBelowPovertyLevel-MOE,MarriedCouplesBelowPovertyLevel,MarriedCouplesBelowPovertyLevel-MOE,PeopleIncomeBelowPovertyLevel,PeopleIncomeBelowPovertyLevel-MOE,ChildrenUnderPovertyLevel,ChildrenUnderPovertyLevel-MOE
0,1400000US17001000100,17001000100,"Census Tract 1, Adams County, Illinois",59.0,4.3,56.5,4.6,2.5,2.1,53.4,...,43.3,7.1,2.0,2.3,2.5,2.8,5.0,2.4,1.1,1.7
1,1400000US17001000201,17001000201,"Census Tract 2.01, Adams County, Illinois",55.1,4.8,52.4,4.9,2.7,1.8,52.8,...,41.2,6.4,15.6,7.6,8.4,7.7,15.6,5.9,21.7,10.8
2,1400000US17001000202,17001000202,"Census Tract 2.02, Adams County, Illinois",76.4,3.7,72.6,4.2,3.8,3.1,75.6,...,31.4,7.5,16.1,10.0,4.0,4.4,19.0,7.5,22.1,14.8
3,1400000US17001000400,17001000400,"Census Tract 4, Adams County, Illinois",53.1,5.6,48.3,5.9,4.8,2.1,51.0,...,56.0,7.0,21.5,8.3,15.1,9.0,30.4,7.5,46.5,17.3
4,1400000US17001000500,17001000500,"Census Tract 5, Adams County, Illinois",53.0,7.2,48.0,6.4,4.9,2.7,44.4,...,54.9,5.5,20.3,9.5,10.3,11.2,26.0,9.8,40.2,18.1


In [3]:
gdf = gpd.read_file("./data/geodata/Chicago_CensusTract.geojson")
gdf.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,FamiliesBelowPovertyLevel-MOE,MarriedCouplesBelowPovertyLevel,MarriedCouplesBelowPovertyLevel-MOE,PeopleIncomeBelowPovertyLevel,PeopleIncomeBelowPovertyLevel-MOE,ChildrenUnderPovertyLevel,ChildrenUnderPovertyLevel-MOE,city,HIVPrevalence2017,geometry
0,17,031,030101,17031030101,301.01,Census Tract 301.01,G5020,S,122359,115695,...,3.1,0.0,5.2,14.0,4.9,0.0,8.8,Chicago,2615,"POLYGON ((-87.66034 41.99229, -87.66011 41.992..."
1,17,031,030701,17031030701,307.01,Census Tract 307.01,G5020,S,69878,0,...,16.0,12.6,18.4,26.2,9.3,50.7,26.2,Chicago,5293,"POLYGON ((-87.66007 41.98362, -87.65984 41.983..."
2,17,031,070103,17031070103,701.03,Census Tract 701.03,G5020,S,69094,0,...,17.6,0.0,17.6,15.6,7.4,0.0,27.3,Chicago,367,"POLYGON ((-87.64422 41.92805, -87.64381 41.928..."
3,17,031,831900,17031831900,8319,Census Tract 8319,G5020,S,324322,0,...,6.4,0.0,6.5,3.8,3.0,0.0,5.6,Chicago,324,"POLYGON ((-87.67365 41.94647, -87.67363 41.946..."
4,17,031,832600,17031832600,8326,Census Tract 8326,G5020,S,721629,0,...,2.1,1.6,2.4,8.4,3.5,4.2,4.5,Chicago,307,"POLYGON ((-87.66421 41.91697, -87.66411 41.916..."


In [4]:
import ipywidgets as widgets

drop = widgets.Dropdown(
    options=[i for i in Chicago_data.columns if i[-4:]!="-MOE"],
    value='ChildrenUnderPovertyLevel',
    description='Property to Map',
    disabled=False)
display(drop)

Dropdown(description='Property to Map', index=57, options=('GEO.id', 'GEOID', 'GEO.display-label', 'LaborForce…

In [5]:
import folium

to_map = drop.value
m = folium.Map(zoom_start=11, location=[41.88, -87.6])
geo_path = "./data/geodata/Chicago_CensusTract.geojson"
Chicago_data["GEOID"]=Chicago_data["GEOID"].astype(int)

folium.Choropleth(
 geo_data=geo_path,
 name='choropleth',
 data=Chicago_data,
 columns=['GEOID', to_map],
 key_on='feature.properties.GEOID',
 fill_color='BuPu',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name=to_map
).add_to(m)
m.save('./maps/Chicago.html')

from IPython.display import IFrame

IFrame(src='./maps/Chicago.html', width=900, height=1000)

In [6]:
df = pd.read_pickle("./data/processed/Cleaned_Chicago_Sales.pkl")
print("Data frame has {} rows and {} columns".format(len(df), len(df.columns)))
df.head()

Data frame has 326484 rows and 170 columns


,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,Rooms,...,Square of Latitude,Log of Longitude,SQRT of Longitude,Square of Longitude,Log of Lot Size Squared,SQRT of Lot Size Squared,Square of Lot Size Squared,Log of Rooms,SQRT of Rooms,Square of Rooms
2,16094150130000,211,13,4500.0,77,3.0,6.0,2.0,2.0,24.0,...,1754.268799,4.474504,9.367557,7700.257324,16.823666,4500.0,4.100625e+14,3.178054,4.898980,576.0
5,4252000820000,204,100,33898.0,25,5.0,0.0,2.0,4.0,11.0,...,1771.669678,4.474834,9.369101,7705.334961,20.862223,33898.0,1.320372e+18,2.397895,3.316625,121.0
8,14322110150000,208,12,3720.0,74,3.0,0.0,2.0,6.0,9.0,...,1757.492310,4.473360,9.362197,7682.651367,16.442959,3720.0,1.915013e+14,2.197225,3.000000,81.0
9,27021200080000,204,34,16079.0,28,1.0,0.0,3.0,1.0,7.0,...,1733.995605,4.475452,9.371996,7714.864746,19.370539,16079.0,6.683995e+16,1.945910,2.645751,49.0
11,13121080620000,204,42,7560.0,71,1.0,0.0,2.0,1.0,6.0,...,1762.405518,4.473950,9.364961,7691.726074,17.861254,7560.0,3.266534e+15,1.791759,2.449490,36.0


In [7]:
to_join = Chicago_data[ [i for i in Chicago_data.columns if i[-4:]!="-MOE"] ]
to_join["Census Tract"] = to_join["GEOID"].astype(str).str[-6:].astype(int)
print(to_join.columns)
to_drop = [ 'GEO.id', 'GEOID', 'GEO.display-label']
for col in to_drop:
    to_join.drop(col, axis=1, inplace=True)
to_join.head()

Index(['GEO.id', 'GEOID', 'GEO.display-label', 'LaborForceParticipationRate',
       'EmploymentRate', 'UnemploymentRate', 'FemaleLaborForceParticipation',
       'FemaleCivilianEmployment', 'CommutingToWork-DrivingAlone',
       'CommutingToWork-Carpool', 'CommutingToWork-PublicTransit',
       'CommutingToWork-PublicTransit.1', 'CommutingToWork-Walked',
       'CommutingToWork-Other', 'CommutingToWork-Home', 'PrivateWageSalary',
       'Income<10,000', 'Income10k-15k', 'Income15k-25k', 'Income25k-35k',
       'Income35k-50k', 'Income50k-75k', 'Income75k-100k', 'Income100k-150k',
       'Income150k-200k', 'Income200K+', 'MedianHouseholdIncome',
       'MeanHouseholdIncome', 'HouseholdsWithEarnings',
       'HouseholdsMeanEarnings', 'HouseholdsWSS', 'HouseholdsMeanSSIncome',
       'HouseholdsWPublicAssistanceIncome',
       'HouseholdsMeanPublicAssistanceIncome',
       'HouseholdsWFoodStampSNAPBenefits', 'FamiliesIncome<10k',
       'FamiliesIncome10k-15k', 'FamiliesIncome15k-25k',
 

<ipython-input-7-b3b4ff43b360>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_join["Census Tract"] = to_join["GEOID"].astype(str).str[-6:].astype(int)
/home/dijkstra/Nextcloud/Documents/UIUC/Y2F/ACE592/Project/env/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,LaborForceParticipationRate,EmploymentRate,UnemploymentRate,FemaleLaborForceParticipation,FemaleCivilianEmployment,CommutingToWork-DrivingAlone,CommutingToWork-Carpool,CommutingToWork-PublicTransit,CommutingToWork-PublicTransit.1,CommutingToWork-Walked,...,NonFamilyHouseholdsMeanIncome,MedianWorkerEarnings,HealthInsuranceCoverage,HealthInsuranceCoverage-Private,HealthInsuranceCoverage-Public,FamiliesBelowPovertyLevel,MarriedCouplesBelowPovertyLevel,PeopleIncomeBelowPovertyLevel,ChildrenUnderPovertyLevel,Census Tract
0,59.0,56.5,2.5,53.4,50.6,81.9,12.3,0.0,1.3,0.6,...,50463.0,34480.0,99.1,76.2,43.3,2.0,2.5,5.0,1.1,100
1,55.1,52.4,2.7,52.8,50.5,79.5,10.1,4.0,4.3,0.5,...,32086.0,26972.0,87.9,63.7,41.2,15.6,8.4,15.6,21.7,201
2,76.4,72.6,3.8,75.6,73.9,79.4,9.8,0.0,2.0,5.5,...,29276.0,28386.0,93.7,72.0,31.4,16.1,4.0,19.0,22.1,202
3,53.1,48.3,4.8,51.0,46.5,73.2,14.9,0.0,2.1,3.9,...,29092.0,25893.0,89.5,43.8,56.0,21.5,15.1,30.4,46.5,400
4,53.0,48.0,4.9,44.4,43.0,82.0,3.2,1.2,1.3,10.6,...,24861.0,22910.0,91.0,45.0,54.9,20.3,10.3,26.0,40.2,500


In [8]:
df["Census Tract"] = df["Census Tract"].astype(float).astype(int)
df = pd.merge(df, to_join, 
         on="Census Tract",
         how="left")
print(len(df))
df.head()

339204


,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,Rooms,...,NonFamilyHouseholdsMedianIncome,NonFamilyHouseholdsMeanIncome,MedianWorkerEarnings,HealthInsuranceCoverage,HealthInsuranceCoverage-Private,HealthInsuranceCoverage-Public,FamiliesBelowPovertyLevel,MarriedCouplesBelowPovertyLevel,PeopleIncomeBelowPovertyLevel,ChildrenUnderPovertyLevel
0,16094150130000,211,13,4500.0,77,3.0,6.0,2.0,2.0,24.0,...,14806.0,20590.0,22252.0,85.2,29.6,60.4,27.7,11.5,34.1,43.9
1,4252000820000,204,100,33898.0,25,5.0,0.0,2.0,4.0,11.0,...,48929.0,106485.0,60333.0,96.2,89.6,21.4,2.6,1.0,1.9,1.7
2,14322110150000,208,12,3720.0,74,3.0,0.0,2.0,6.0,9.0,...,82589.0,97714.0,36996.0,95.8,92.8,9.0,0.0,0.0,12.4,0.0
3,27021200080000,204,34,16079.0,28,1.0,0.0,3.0,1.0,7.0,...,40882.0,54363.0,50197.0,96.2,85.5,25.1,2.6,3.0,5.6,5.7
4,13121080620000,204,42,7560.0,71,1.0,0.0,2.0,1.0,6.0,...,77222.0,77490.0,45552.0,95.3,82.7,21.4,1.6,0.0,2.9,0.0


In [9]:
print(len(df))
test = df[df["ChildrenUnderPovertyLevel"].isnull()]
print(len(test))
test["Census Tract"]

339204
0


Series([], Name: Census Tract, dtype: int64)

In [10]:
for col in df.columns:
    if df[col].dtypes == np.int64:
        #print("int64")
        df[col] = df[col].astype("int32")
    elif df[col].dtypes == np.float64:
        df[col] = df[col].astype("float32")
        #print("float64")
    print("Col: {:35s}, Dtype: {}".format(col, df[col].dtypes))
print("Writing out CSV and Pickle with {} rows".format(len(df)))
df.to_pickle("./data/processed/Cleaned_Chicago_Sales_wACS.pkl")
df.to_csv("./data/processed/Cleaned_Chicago_Sales_wACS.csv")

Col: PIN                                , Dtype: int32
Col: Property Class                     , Dtype: int32
Col: Neighborhood Code                  , Dtype: int32
Col: Land Square Feet                   , Dtype: float32
Col: Town Code                          , Dtype: int32
Col: Type of Residence                  , Dtype: float32
Col: Apartments                         , Dtype: float32
Col: Wall Material                      , Dtype: float32
Col: Roof Material                      , Dtype: float32
Col: Rooms                              , Dtype: float32
Col: Bedrooms                           , Dtype: float32
Col: Basement                           , Dtype: float32
Col: Basement Finish                    , Dtype: float32
Col: Central Heating                    , Dtype: float32
Col: Central Air                        , Dtype: float32
Col: Fireplaces                         , Dtype: float32
Col: Attic Type                         , Dtype: float32
Col: Attic Finish                      

Col: LaborForceParticipationRate        , Dtype: float32
Col: EmploymentRate                     , Dtype: float32
Col: UnemploymentRate                   , Dtype: float32
Col: FemaleLaborForceParticipation      , Dtype: float32
Col: FemaleCivilianEmployment           , Dtype: float32
Col: CommutingToWork-DrivingAlone       , Dtype: float32
Col: CommutingToWork-Carpool            , Dtype: float32
Col: CommutingToWork-PublicTransit      , Dtype: float32
Col: CommutingToWork-PublicTransit.1    , Dtype: float32
Col: CommutingToWork-Walked             , Dtype: float32
Col: CommutingToWork-Other              , Dtype: float32
Col: CommutingToWork-Home               , Dtype: float32
Col: PrivateWageSalary                  , Dtype: float32
Col: Income<10,000                      , Dtype: float32
Col: Income10k-15k                      , Dtype: float32
Col: Income15k-25k                      , Dtype: float32
Col: Income25k-35k                      , Dtype: float32
Col: Income35k-50k             